**This notebook presents the tic-tac-toe in NAHUATL using the "tyoc213/wav2vec2-large-xlsr-nahuatl" model using a trained SVM and a vocabulary filter**

# Install the library of huggingsound and import the Speech recognition model of PatrickVonPlaten

In [ ]:
# Install speech recognition from huggingsound
!pip install huggingsound
from huggingsound import SpeechRecognitionModel

In [ ]:
# Upload model from huggingface
model = SpeechRecognitionModel("tyoc213/wav2vec2-large-xlsr-nahuatl")
#tyoc213/wav2vec2-large-xlsr-nahuatl

# Code for audio collection  

In [ ]:
# Add all the code to obtain the audio from microphone
!pip install ffmpeg-python
from scipy.io.wavfile import write

In [ ]:
# To record in Google Collab with the microphone, we lost the original sourse, but here
# is a secondary one https://stackoverflow.com/questions/67709421/how-to-transcribe-the-recording-for-speech-recognization
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
# Test that the michophone audio function works
# audio, sr = get_audio()
# write("example.wav", 44100, audio)

# Upload SVM model

In [ ]:
# Upload the Linear Vector Machine trained from the notebook "CLASIFICACION_NAHUATL.ipynb" as a 'svc.pkl' file, to recognize nahuatl numbers
from sklearn.svm import LinearSVC
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from joblib import dump, load
#dump(clf, 'filename.joblib') 
svc = load('/content/svc.pkl')

# Filter Function

In [ ]:
# Define the filter function, which reads both the original nahuatl speech model and the Vector Support Machine output 

def filter3(written_value, predicted_value):

  predicted_value_orig = predicted_value

  written_value = written_value.strip()
  predicted_value = predicted_value.strip()

  if (('ie' in written_value) or ('ei' in written_value) or ('yi' in written_value) or ('ye' in written_value) or ('ey' in written_value)) and (len(written_value)<=5) and (predicted_value != 'uno')and (predicted_value != 'dos'):
    return 'tres'
  elif ((written_value[:2] == 'ma') ):
    return 'cinco'
  elif ((written_value[:2] == 'na') ):
    return 'cuatro'
  elif (('ie' in written_value) or ('ei' in written_value) or ('yi' in written_value) or ('ye' in written_value) or ('ey' in written_value)) and (len(written_value)>5) and (predicted_value != 'uno')and (predicted_value != 'dos') and (written_value[:2] == 'ch'):
    return 'ocho'
  elif ((written_value[0] == 'm') and ('l' in written_value)):
    return 'cinco'
  elif (('ce' in written_value) or ('se' in written_value)) and (len(written_value)>5) and (predicted_value != 'uno') and (written_value[:2] == 'ch'):
    return 'seis'
  else:
    return predicted_value_orig

# Game Demonstration

In [ ]:
#Read libraries to run the tic tac toe game demonstration (also called gato (cat) in spanish  and miztli with a direct translation)
!pip install scipy
!pip install NumPy
!pip install spa2num
from spa2num.converter import to_number

     |████████████████████████████████| 101 kB 5.8 MB/s 


In [ ]:
# Helper HTML code
RESTART_HTML = """
<script>

//document.body.appendChild(my_div);
document.body.removeChild(document.body.lastElementChild); 

</script>
"""

In [ ]:
# Adapted the tic-tac-toe code from https://gist.github.com/rnovec/946d65e47b44cff5a060bb4dbe890ce2 
# The speech button runs automatically and must be stopped to take another turn
# Answer the  number (in NAHUATL) of the cell with the convention:

# 7|8|9
# -+-+-
# 4|5|6
# -+-+-
# 1|2|3

theBoard = {'7': ' ' , '8': ' ' , '9': ' ' ,
            '4': ' ' , '5': ' ' , '6': ' ' ,
            '1': ' ' , '2': ' ' , '3': ' ' }
def printBoard(board):
    print(board['7'] + '|' + board['8'] + '|' + board['9'])
    print('-+-+-')
    print(board['4'] + '|' + board['5'] + '|' + board['6'])
    print('-+-+-')
    print(board['1'] + '|' + board['2'] + '|' + board['3'])
    
board_keys = []

for key in theBoard:
    board_keys.append(key)

# Now we'll write the main function which has all the gameplay functionality.
def game():

    turn = 'X'
    count = 0


    for i in range(10):
        printBoard(theBoard)
        print("Es tu turno," + turn + ". Cuál es tu movimiento? \n")
        
        freq = 16000

        # Record audio
        audio, sr = get_audio()
        write("recordingprueba.wav", 44100, audio)

        # Convert the NumPy array to audio file
        audio_paths = ["./recordingprueba.wav"]
        #model.transcribe(audio_paths)
        b=model.transcribe(audio_paths)[0]
        first_value = list(b.values())[0]

        print('\n written value:   ' + first_value)

        # print(first_value[:1])
        # print(first_value[:1] == 'ma')

        predicted_value = svc.predict([first_value])[0]

        print('\n predicted value:   ' + predicted_value)

        filtered_value = filter3(first_value, predicted_value)

        print('\n filtered value:   ' + filtered_value)

        move = str(to_number(filtered_value))               

        print(move)

        if theBoard[move] == ' ':
            theBoard[move] = turn
            count += 1
        else:
            print("La celda está ocupada.\n Vuelve a tirar")
            continue

        # Now we will check if player X or O has won,for every move after 5 moves. 
        if count >= 5:
            if theBoard['7'] == theBoard['8'] == theBoard['9'] != ' ': # across the top
                printBoard(theBoard)
                print("\n Juego terminado .\n")                
                print(" ** " +turn + " ganó. **")                
                break
            elif theBoard['4'] == theBoard['5'] == theBoard['6'] != ' ': # across the middle
                printBoard(theBoard)
                print("\n Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break
            elif theBoard['1'] == theBoard['2'] == theBoard['3'] != ' ': # across the bottom
                printBoard(theBoard)
                print("\n Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break
            elif theBoard['1'] == theBoard['4'] == theBoard['7'] != ' ': # down the left side
                printBoard(theBoard)
                print("\n Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break
            elif theBoard['2'] == theBoard['5'] == theBoard['8'] != ' ': # down the middle
                printBoard(theBoard)
                print("\n Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break
            elif theBoard['3'] == theBoard['6'] == theBoard['9'] != ' ': # down the right side
                printBoard(theBoard)
                print("\n Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break 
            elif theBoard['7'] == theBoard['5'] == theBoard['3'] != ' ': # diagonal
                printBoard(theBoard)
                print("\n  Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break
            elif theBoard['1'] == theBoard['5'] == theBoard['9'] != ' ': # diagonal
                printBoard(theBoard)
                print("\n Juego terminado.\n")                
                print(" ** " +turn + " ganó. **")
                break 

        # If neither X nor O wins and the board is full, we'll declare the result as 'tie'.
        if count == 9:
            print("\n Juego terminado.\n")                
            print("Es un empate!!")

        # Now we have to change the player after every move.
        if turn =='X':
            turn = 'O'
        else:
            turn = 'X'       

        # if i>0:
        display(HTML(RESTART_HTML)) 
    
    # Now we will ask if player wants to restart the game or not.
    restart = input("Quieres volver a jugar?(Si/No)")
    if restart == "Si" or restart == "U":  
        for key in board_keys:
            theBoard[key] = " "

        game()

if __name__ == "__main__":
    game()

 | | 
-+-+-
 | | 
-+-+-
 | | 
Es tu turno,X. Cuál es tu movimiento? 



100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


 written value:   ome

 predicted value:   dos

 filtered value:   dos
2


 | | 
-+-+-
 | | 
-+-+-
 |X| 
Es tu turno,O. Cuál es tu movimiento? 



100%|██████████| 1/1 [00:03<00:00,  3.85s/it]

...


 written value:   naweh

 predicted value:   nueve

 filtered value:   cuatro
4


 |O|X
-+-+-
O|O| 
-+-+-
X| |X
Es tu turno,X. Cuál es tu movimiento? 



100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


 written value:   

 predicted value:   nueve


IndexError: ignored